In [1]:
pip install surprise


You should consider upgrading via the 'c:\users\checkout\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.pyplot import figure
from PIL import Image
import requests
from io import BytesIO
import os
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from scipy import sparse
import re

import warnings
warnings.simplefilter("ignore")

In [3]:
url = "https://raw.githubusercontent.com/jayasurya17/cmpe-256-term-project/master/clothing_data_processed.csv"
clothing_data_processed = pd.read_csv(url)
clothing_data_processed

,fit,user_id,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,bust_size,bust_type
0,fit,420272,2260466,137.000000,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,M,28.0,34.0,d
1,fit,273551,153475,132.000000,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,M,36.0,34.0,b
2,fit,360448,1063761,137.391709,10.0,party,This hugged in all the right places! It was a ...,petite,It was a great time to celebrate the (almost) ...,shift,64.0,XS,32.0,34.0,c
3,fit,909926,126335,135.000000,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,S,34.0,34.0,c
4,fit,151944,616682,145.000000,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,M,27.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,2252812,140.000000,10.0,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,69.0,S,42.0,34.0,dd
192540,fit,118398,682043,100.000000,10.0,work,The pattern contrast on this dress is really s...,petite,LOVE it!,dress,61.0,XS,29.0,32.0,c
192541,fit,47002,683251,135.000000,6.0,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",dress,68.0,S,31.0,36.0,a
192542,fit,961120,126335,165.000000,10.0,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,dress,66.0,L,31.0,36.0,c


In [4]:
#preprocessed training dataset
url1 = "https://raw.githubusercontent.com/jayasurya17/cmpe-256-term-project/master/train.csv"
train_data = pd.read_csv(url1)
train_data

,fit,user_id,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,bust_size,bust_type
0,small,379417,658706,120.000000,10.0,party,"It ran a little small, in my opinion, but the ...",straight & narrow,"Perfect, Unique Dress!",dress,65.0,M,34.0,34.0,b
1,fit,904971,182915,120.000000,10.0,party,Wore this to a Christmas party - so comfortabl...,straight & narrow,Great fit and material!,dress,64.0,S,52.0,34.0,b
2,small,163215,1979533,150.000000,10.0,other,As the temperatures climbed into the 90s yeste...,hourglass,Bright and fun for engagement pics!,dress,69.0,S,28.0,34.0,d
3,small,232640,880935,140.000000,8.0,wedding,The dress color was a wow! I found the dress ...,hourglass,This dress is a standout!,dress,68.0,M,38.0,34.0,c
4,fit,691117,180014,137.391709,10.0,party,"This dress is very flattering, and the cutouts...",athletic,So easy to wear and comfy! Paired with red acc...,dress,69.0,L,33.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154030,small,733207,2358935,128.000000,8.0,work,This was a cute top. I really liked the lace a...,athletic,Fun Top.,top,66.0,S,38.0,32.0,dd
154031,fit,873762,625911,160.000000,10.0,wedding,This dress was great! The style of the dress i...,athletic,"Classic design, with an edge.",dress,68.0,XL,27.0,36.0,b
154032,fit,14227,1515649,145.000000,8.0,party,no complaints,pear,fit like a glove,dress,65.0,M,37.0,32.0,d
154033,fit,421813,127081,170.000000,8.0,wedding,The fit of the dress is pretty true to size. I...,hourglass,Cancelled wedding. but gorgeous dress,gown,66.0,XL,34.0,36.0,d


In [5]:
#preprocessed test dataset
url2 = "https://raw.githubusercontent.com/jayasurya17/cmpe-256-term-project/master/test.csv"
test_data = pd.read_csv(url2)
test_data

,fit,user_id,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,bust_size,bust_type
0,fit,492205,208647,115.000000,8.0,wedding,"I rented this dress for my mom, she loved it! ...",straight & narrow,Great Dress for a wedding!,dress,66.0,XL,25.0,32.0,d
1,fit,596658,145906,135.000000,8.0,wedding,"I was at a winter, black tie wedding, a lot of...",athletic,Received a lot of compliments,gown,65.0,S,31.0,36.0,c
2,fit,88342,1740815,135.000000,10.0,work,Fabulous black & white dress by Carven! We lov...,hourglass,Stylist Review!,dress,65.0,XL,29.0,34.0,c
3,fit,501057,2803807,110.000000,10.0,everyday,good fit,petite,looks great,jacket,63.0,XS,34.0,30.0,b
4,fit,662466,2521411,137.391709,8.0,party,It's a cute jacket that I wore a few times - b...,straight & narrow,Cute but not amazing,jacket,68.0,S,27.0,34.0,c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38504,fit,884008,172027,105.000000,10.0,date,Slightly loose in the top but was absolutely f...,petite,Amazing outfit to go see Hamilton!,dress,64.0,XS,31.0,34.0,a
38505,large,437160,130259,137.391709,8.0,wedding,Dress was a beautiful color but it just didn't...,full bust,Didn't work out:(,dress,62.0,L,50.0,34.0,c
38506,fit,425211,416213,137.391709,8.0,wedding,Open back means no bra (which I usually never ...,full bust,Perfect for a fall wedding!,gown,62.0,XL,30.0,38.0,d
38507,large,970560,1146825,123.000000,10.0,everyday,"Lose fit, Very comfortable, Wear any color sh...",petite,"Overall the dress, although a little lose fitt...",dress,64.0,XS,36.0,32.0,a


In [6]:
#for the train data - list out all users to avoid repetation and place an index to each user.
user_ids_train = train_data["user_id"].unique().tolist()

#index all the unique users
user2user_encoded_train = {x: i for i, x in enumerate(user_ids_train)}
#print(user2user_encoded_train)

userencoded2user_train = {i: x for i, x in enumerate(user_ids_train)}
#print(userencoded2user_train)

In [7]:
#for the test data - list out all users to avoid repetation and place an index to each user.
user_ids_test = test_data["user_id"].unique().tolist()

#index all the unique users
user2user_encoded_test = {x: i for i, x in enumerate(user_ids_test)}
#print(user2user_encoded_test)

userencoded2user_test = {i: x for i, x in enumerate(user_ids_test)}
#print(userencoded2user_test)

In [8]:
#for the train data - list out all items, to avoid repetation and place an index to each items.
item_ids_train = train_data["item_id"].unique().tolist()

item2item_encoded_train = {x: i for i, x in enumerate(item_ids_train)}
#print(item2item_encoded_train)

itemencoded2item_train = {i: x for i, x in enumerate(item_ids_train)}
#print(itemencoded2item_train)

In [9]:
#for the test data - list out all items, to avoid repetation and place an index to each items.
item_ids_test = test_data["item_id"].unique().tolist()

item2item_encoded_test = {x: i for i, x in enumerate(item_ids_test)}
#print(item2item_encoded_test)

itemencoded2item_test= {i: x for i, x in enumerate(item_ids_test)}
#print(itemencoded2item_test)

In [10]:
#mapping back to the train_data dataframe
train_data["userfind"] = train_data["user_id"].map(user2user_encoded_train)
train_data["itemfind"] = train_data["item_id"].map(item2item_encoded_train)
print (train_data)

          fit  user_id  item_id      weight  rating rented for  \
0       small   379417   658706  120.000000    10.0      party   
1         fit   904971   182915  120.000000    10.0      party   
2       small   163215  1979533  150.000000    10.0      other   
3       small   232640   880935  140.000000     8.0    wedding   
4         fit   691117   180014  137.391709    10.0      party   
...       ...      ...      ...         ...     ...        ...   
154030  small   733207  2358935  128.000000     8.0       work   
154031    fit   873762   625911  160.000000    10.0    wedding   
154032    fit    14227  1515649  145.000000     8.0      party   
154033    fit   421813   127081  170.000000     8.0    wedding   
154034    fit   902767   126335  155.000000     8.0      party   

                                              review_text          body type  \
0       It ran a little small, in my opinion, but the ...  straight & narrow   
1       Wore this to a Christmas party - so com

In [11]:
#mapping back to the test_data dataframe
test_data["userfind"] = test_data["user_id"].map(user2user_encoded_test)
test_data["itemfind"] = test_data["item_id"].map(item2item_encoded_test)
print (test_data)

         fit  user_id  item_id      weight  rating     rented for  \
0        fit   492205   208647  115.000000     8.0        wedding   
1        fit   596658   145906  135.000000     8.0        wedding   
2        fit    88342  1740815  135.000000    10.0           work   
3        fit   501057  2803807  110.000000    10.0       everyday   
4        fit   662466  2521411  137.391709     8.0          party   
...      ...      ...      ...         ...     ...            ...   
38504    fit   884008   172027  105.000000    10.0           date   
38505  large   437160   130259  137.391709     8.0        wedding   
38506    fit   425211   416213  137.391709     8.0        wedding   
38507  large   970560  1146825  123.000000    10.0       everyday   
38508  small    30013   132738  142.000000    10.0  formal affair   

                                             review_text          body type  \
0      I rented this dress for my mom, she loved it! ...  straight & narrow   
1      I was 

In [12]:
#preparing train set
num_users_train = len(user2user_encoded_train)
num_items_train = len(itemencoded2item_train)

train_data["rating"] = train_data["rating"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = min(train_data["rating"])
max_rating = max(train_data["rating"])

print(
    "Number of users in train data: {}, Number of items in train data: {}, Min rating: {}, Max rating: {}".
     format(num_users_train, num_items_train, min_rating, max_rating)
)

Number of users in train data: 90413, Number of items in train data: 5762, Min rating: 2.0, Max rating: 10.0


In [13]:
#Prepare training and validation dataset

x = train_data[["userfind", "itemfind"]].values

# Normalize the targets between 0 and 1. Makes it easy to train.
y = train_data["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values

# Assuming training on 80% of the data and validating on 20%.
train_indices = int(0.80 * train_data.shape[0])  #take 80% of the rows as training data

x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

### Neural network based CF

In [14]:
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly

import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import accuracy

ModuleNotFoundError: No module named 'tensorflow'

https://towardsdatascience.com/movie-recommender-system-a5dbfdb2585d

he_normal> It draws samples from a truncated normal distribution centered on 0 with stddev = sqrt(2 / fan_in) where fan_in is the number of input units in the weight tensor.

EMBEDDING_SIZE>  Embedding is the concept of mapping from discrete objects such as words to vectors and real numbers. 
Keras offers an Embedding layer that can be used for neural networks on text data. It requires that the input data be integer encoded, so that each word is represented by a unique integer. This data preparation step can be performed using the Tokenizer API also provided with Keras.
There are a few different embedding vector sizes, including 50, 100, 200 and 300 dimensions. You can download this collection of embeddings and we can seed the Keras Embedding layer with weights from the pre-trained embedding for the words in your training dataset.

In [ ]:
#defined the embedding layer size
EMBEDDING_SIZE = 100


class RecommenderNet(keras.Model):
    def __init__(self, num_users, num_items, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_size = embedding_size
        #doing embedding on user
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        
        
        #doing embedding on item
        self.items_embedding = layers.Embedding(
            num_items,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-6),
        )
        self.items_bias = layers.Embedding(num_items, 1)

        
    def call(self, inputs):
        #preparing user vectors and item vectors
        user_vector = self.user_embedding(inputs[:, 0]) 
        user_bias = self.user_bias(inputs[:, 0])  
        items_vector = self.items_embedding(inputs[:, 1])
        items_bias = self.items_bias(inputs[:, 1])  
        
        #dot product of vectors
        dot_user_items = tf.tensordot(user_vector, items_vector, 2)
        
        x = dot_user_items + user_bias + items_bias
        # normalized rating
        return tf.nn.sigmoid(x)

model = RecommenderNet(num_users_train, num_items_train, EMBEDDING_SIZE)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=0.001)
)

In [ ]:
#calculating the loss
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=100,
    epochs=5,
    verbose=1,
    validation_data=(x_val, y_val),
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()


In [ ]:
#evaluation by MSE, MAE
model = RecommenderNet(num_users_train, num_items_train, EMBEDDING_SIZE)
model.compile(
    keras.optimizers.Adam(lr=0.001), loss="mean_squared_error",
    metrics=["mean_absolute_error", "mean_squared_error"])

In [ ]:
#calculating the errors
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=100,
    epochs=5,
    verbose=1,
    validation_data=(x_val, y_val),
)


In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

 # Selected an user from test data and recommended items 

In [ ]:
# what a user has already reviewed

#user_selected = test_data["user_id"] == '492205'
user_selected = test_data.user_id.sample(1).iloc[0]  

items_reviewed_by_user = test_data[test_data.user_id == user_selected]
print(items_reviewed_by_user)

In [ ]:
#what a user has not already reviewed
items__not_reviewed = test_data[~test_data["item_id"].isin(items_reviewed_by_user.item_id.values)]["item_id"]

items__not_reviewed = list(set(items__not_reviewed).intersection(set(item2item_encoded_test.keys())))

items__not_reviewed = [[item2item_encoded_test.get(x)] for x in items__not_reviewed]

user_encoder = user2user_encoded_test.get(user_selected)

user_item_array = np.hstack(([[user_encoder]] * len(items__not_reviewed), items__not_reviewed))  #user_item_array consists of selected_user in userfind and itemfind

rate = model.predict(user_item_array)
ratings = model.predict(user_item_array).flatten()


#For any iterable in python [-10:] denotes the indexing of last 10 items of that iterable.
#[::-1] denotes same list in reverse order
#returns top 10 max ratings' indices
top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_item_ids = [
    itemencoded2item_test.get(items__not_reviewed[x][0]) for x in top_ratings_indices
]

In [ ]:
print(user_item_array)
print(rate)

In [ ]:
print (top_ratings_indices)

In [ ]:
#recommended items 
print("Showing recommendations for user: {}".format(user_selected))

print("Top 10 items recommendations")

recommended_items = test_data[test_data["item_id"].isin(recommended_item_ids)]
recommended_items = recommended_items.drop_duplicates(subset = ["item_id"])

for row in recommended_items.itertuples():
    print(row.item_id)